# Generación de una capa optimizada para etiquetado en capas con alta densidad de entidades

En proyectos de análisis espacial es común trabajar con capas que contienen un número muy elevado de entidades, como sucede con la zonificación secundaria a nivel municipal. Al intentar visualizar etiquetas para cada polígono, estas suelen superponerse, dificultando la lectura y reduciendo la utilidad del mapa.

Para resolver este problema, se implementa un proceso que agrupe espacialmente las entidades cercanas dentro de una distancia específica, generando una capa simplificada que permita colocar etiquetas de manera clara y ordenada.

## Objetivo del proceso
Crear una nueva capa con geometrías agrupadas según su proximidad, de modo que:
- Las etiquetas no se traslapen.
- Se reduzca el número total de entidades a etiquetar.
- Se mantenga la correspondencia entre grupos de polígonos y los valores de uso del suelo.

### Descripción general del método

El flujo de trabajo considera los siguientes pasos:

- Identificar la columna de interés <br>
Se elige el atributo que se utilizará como etiqueta (por ejemplo, cve_uso).
- Asignar una distancia de búsqueda por categoría<br>
Cada grupo de usos del suelo recibe un radio de agrupamiento adecuado según su naturaleza.
- Generar buffers individuales<br>
Para cada entidad se crea un buffer con la distancia correspondiente a su categoría.
- Disolver los buffers por categoría<br>
Esto unifica las áreas cercanas y reduce el número de geometrías, creando un polígono multiparte.
- Convertir a geometrías de una sola parte (singlepart) <br>
Se separan nuevamente los polígonos en partes individuales, pero ahora agrupadas por proximidad.
- Crear la capa final consolidada<br>

Todas las categorías procesadas se combinan en una sola capa que contiene:
- Menos entidades.
- Geometrías agrupadas.
- Distancias aplicadas para control de etiquetado.
<br>
El resultado es una capa optimizada para la colocación de etiquetas, evitando saturación visual y mejorando la legibilidad del mapa final.

In [ ]:
import geopandas as gpd
import pandas as pd
import os

# Distancia distinta por uso

In [2]:
# === CONFIGURACIÓN ===
input_shp = 'G:/My Drive/Trabajo/Huixquilucan/02_capas/E-02_Zonificación Secundaria/C15037_zonificacion-secundaria_PMDU_2025.shp'
output_folder = 'G:/My Drive/Trabajo/Huixquilucan/02_capas/E-02_Zonificación Secundaria/'
os.makedirs(output_folder, exist_ok=True)

columna = 'cve_uso'

# Leer el archivo shapefile
gdf = gpd.read_file(input_shp)
# Obtener los valores únicos de la columna 'cve_uso'
valores_uso = gdf[columna].unique()

In [8]:
sorted(gdf['cve_uso'].unique())


['AG-BP',
 'AV',
 'C',
 'CA',
 'CRU 200',
 'CS',
 'CU',
 'CU 2000 MX',
 'CUIM',
 'E',
 'E-ASU',
 'E-CA',
 'E-EC',
 'E-ET',
 'E-RD',
 'E-SAS',
 'F',
 'HM 50',
 'HP',
 'HP 1000 A',
 'HP 1400 A',
 'HP 1500 A',
 'HP 2000 A',
 'HP 2500 A',
 'HP 300 A',
 'HP 4000 A',
 'HP 500 A',
 'HP 800 A',
 'HU 200',
 'HU 200 MX',
 'HU 300',
 'HU 400 MX',
 'IL',
 'NAT-PD',
 'NAT-PE',
 'PR',
 'PT',
 'SUA',
 'SUA-C',
 'SUB',
 'UE']

In [ ]:
# == Definición de distancias por categoría ===
dist_habitacional = 5
dist_equipamiento = -1
dist_comercial = 2
dist_productivo = 5
dist_natural = 15
dist_averde = 20
dist_industrial = 10

# === Grupos de usos por categoría ===
usos_habitacionales = [
    'HM 50',
    'HP','HP 1000 A', 'HP 1400 A','HP 1500 A','HP 2000 A','HP 2500 A','HP 300 A','HP 4000 A', 'HP 500 A','HP 800 A',
    'HU 200','HU 200 MX','HU 300','HU 400 MX'
    ]

usos_equipamientos = [
    'E', 'E-ASU', 'E-CA', 'E-EC', 'E-ET', 'E-RD', 'E-SAS',]

usos_comerciales = [
    'C', 'CS', 'CU', 'CUIM', 'CRU 200', 'PT', 'CA', 'SUB', 'SUA', 'SUA-C', 'UE'
]

usos_productivos = [
    'AG-BP'
]

usos_naturales = [
    'NAT-PE', 'NAT-PD', 'PR', 'F'
]

usos_averde = [
    'AV', 
]

usos_industriales = [
    'IL'
]

# === Construcción automática del diccionario de distancias ===
distancias = {}

# Cargar los grupos en el diccionario principal
for uso in usos_habitacionales:
    distancias[uso] = dist_habitacional

for uso in usos_equipamientos:
    distancias[uso] = dist_equipamiento

for uso in usos_comerciales:
    distancias[uso] = dist_comercial

for uso in usos_productivos:
    distancias[uso] = dist_productivo

for uso in usos_naturales:
    distancias[uso] = dist_natural

for uso in usos_averde:
    distancias[uso] = dist_averde

for uso in usos_industriales:
    distancias[uso] = dist_industrial

In [10]:
# Lista para guardar todos los resultados
gdfs_resultado = []

# === PROCESAMIENTO ===
for valor in valores_uso:
    # Obtener la distancia correspondiente o usar una por defecto
    distancia = distancias.get(valor, 1)
    print(f'Procesando {valor} con buffer de {distancia} m')

    # Filtrar por valor
    gdf_filtrado = gdf[gdf[columna] == valor]

    # Aplicar buffer
    gdf_buffer = gdf_filtrado.copy()
    gdf_buffer['geometry'] = gdf_buffer.buffer(distancia)

    # Disolver y convertir a singlepart
    gdf_disuelto = gdf_buffer.dissolve(by=columna).reset_index()
    gdf_singlepart = gdf_disuelto.explode(index_parts=False)
    gdf_singlepart['dist_buffer'] = distancia

    # Agregar a resultados
    gdfs_resultado.append(gdf_singlepart)

# === CONCATENAR Y GUARDAR ===
gdf_consolidado = gpd.GeoDataFrame(pd.concat(gdfs_resultado, ignore_index=True))
output_total = os.path.join(output_folder, 'C15037_zonificacion-secundaria-etiquetas_PMDU_2025.shp')
gdf_consolidado.to_file(output_total)
print(f'Archivo consolidado exportado: {output_total}')


Procesando HU 200 MX con buffer de 5 m
Procesando E-EC con buffer de -1 m
Procesando E-SAS con buffer de -1 m
Procesando AV con buffer de 20 m
Procesando E-RD con buffer de -1 m
Procesando E-ASU con buffer de -1 m
Procesando E-ET con buffer de -1 m
Procesando E-CA con buffer de -1 m
Procesando CRU 200 con buffer de 2 m
Procesando C con buffer de 2 m
Procesando PR con buffer de 15 m
Procesando NAT-PE con buffer de 15 m
Procesando E con buffer de -1 m
Procesando AG-BP con buffer de 5 m
Procesando IL con buffer de 10 m
Procesando HP 2000 A con buffer de 5 m
Procesando HU 300 con buffer de 5 m
Procesando SUB con buffer de 2 m
Procesando HP 1000 A con buffer de 5 m
Procesando HM 50 con buffer de 5 m
Procesando HU 200 con buffer de 5 m
Procesando F con buffer de 15 m
Procesando HP 800 A con buffer de 5 m
Procesando HP 300 A con buffer de 5 m
Procesando HP 500 A con buffer de 5 m
Procesando HP con buffer de 5 m
Procesando SUA con buffer de 2 m
Procesando HP 1500 A con buffer de 5 m
Procesando

C:\Users\aloac\AppData\Local\Temp\ipykernel_30260\620966538.py:28: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  gdf_consolidado.to_file(output_total)
c:\Users\aloac\miniconda3\envs\gds\Lib\site-packages\pyogrio\raw.py:723: RuntimeWarning: Normalized/laundered field name: 'dist_buffer' to 'dist_buffe'
  ogr_write(


Archivo consolidado exportado: G:/My Drive/Trabajo/Huixquilucan/02_capas/E-02_Zonificación Secundaria/C15037_zonificacion-secundaria-etiquetas_PMDU_2025.shp


# Todo en un solo bloque

In [ ]:
# Ruta de entrada y salida
input_shp = 'G:/My Drive/Trabajo/Huixquilucan/02_capas/E-02_Zonificación Secundaria/C15037_zonificacion-secundaria_PMDU_2025.shp'
output_folder = 'G:/My Drive/Trabajo/huix_insumos/ejercicio_etiquetas'  # Carpeta de salida
# Crear carpeta de salida si no existe
os.makedirs(output_folder, exist_ok=True)
# Columna de interes
columna = 'cve_uso'
# Buffer de distancia
distancia = 1
# Lista para guardar todos los resultados
gdfs_resultado = []
# Leer el archivo shapefile
gdf = gpd.read_file(input_shp)
# Obtener los valores únicos de la columna 'cve_uso'
valores_uso = gdf[columna].unique()
# por cada valor único, procesar
# Iterar sobre cada valor único en la columna 'cve_uso'
# Crea un buffer a N distancia
# Disuelve todas las geometrías en una sola por cve_uso
# Convierte multiparte a singlepart

for valor in valores_uso:
    # Filtrar el GeoDataFrame por el valor de columna
    gdf_filtrado = gdf[gdf[columna] == valor]
    
    # Aplicar buffer 
    gdf_buffer = gdf_filtrado.copy()
    gdf_buffer['geometry'] = gdf_buffer.buffer(distancia)

    # Disolver las geometrías en una sola por cve_uso
    gdf_disuelto = gdf_buffer.dissolve(by=columna)

    # Reiniciar índice para que tenga una columna de identificación
    gdf_disuelto = gdf_disuelto.reset_index()

    # Convertir multiparte a singlepart (explode)
    gdf_singlepart = gdf_disuelto.explode(index_parts=False)
    
    # Agregar a la lista de resultados
    gdfs_resultado.append(gdf_singlepart)

    # Guardar archivo individual
    #output_path = os.path.join(output_folder, f'{valor}_buffer_{distancia}m.shp')
    #gdf_singlepart.to_file(output_path)
    #print(f'Archivo exportado: {output_path}')
    
# Concatenar todos los resultados en un solo GeoDataFrame
gdf_consolidado = gpd.GeoDataFrame(pd.concat(gdfs_resultado, ignore_index=True))

# Guardar archivo consolidado
output_total = os.path.join(output_folder, f'consolidado_buffer_{distancia}m.shp')
gdf_consolidado.to_file(output_total)
print(f'Archivo consolidado exportado: {output_total}')